# Extract ONT QC Data<a class="tocSkip">

**This notebook reads in data from NTSM and Coverage WDLS (stored in data tables). This is part of the ONT QC process.**

**Below are the steps taken in this notebook:**
1. Import Statements & Global Variable Definitions
2. Define Functions
3. Read In Sample Names
4. Create Dataframe Of Files
5. Examine results

**Note**: These results are not written back to the data tables or to files

# Import Statements & Global Variable Definitions

## Installs

In [1]:
## May need to restart kernel after the following installs 

In [2]:
%%capture
%pip install gcsfs
## capture CANNOT have comments above it
## For reading CSVs stored in Google Cloud (without downloading them first)
## May need to restart kernel after install 

In [3]:
%%capture
%pip install --upgrade --no-cache-dir --force-reinstall terra-pandas
%pip install --upgrade --no-cache-dir  --force-reinstall git+https://github.com/DataBiosphere/terra-notebook-utils
## For reading/writing data tables into pandas data frames
## May need to restart kernel after install 

## Import Statements

In [1]:
from firecloud import fiss
import pandas as pd 
import numpy as np
import terra_pandas as tp
import os                 
import subprocess       
import re                 
import io
import gcsfs
import glob

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from typing import Any, Callable, List, Optional
from terra_notebook_utils import table, WORKSPACE_NAME, WORKSPACE_GOOGLE_PROJECT


## Global Variable Declarations

In [2]:
# Get the Google billing project name and workspace name for current workspace
PROJECT = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE =os.path.basename(os.path.dirname(os.getcwd()))
bucket = os.environ['WORKSPACE_BUCKET'] + "/"

# Verify that we've captured the environment variables
print("Billing project: " + PROJECT)
print("Workspace: " + WORKSPACE)
print("Workspace storage bucket: " + bucket)

Billing project: human-pangenome-ucsc
Workspace: HPRC_YEAR3_ONT_GUPPY6
Workspace storage bucket: gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/


# Extract NTSM Data

## Read in NTSM Data Table

In [3]:
ntsm_df = tp.table_to_dataframe("ntsm", workspace=WORKSPACE, workspace_namespace=PROJECT)

ntsm_df.head()

,ntsv_count_2,fastq_list,ntsm_eval_out,read_2_fastq,1000G,read_1_fastq,sample,ONT_pass_bam,ntsv_count_1
ntsm_id,,,,,,,,,
0,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,[gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,NA18522,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,GM18522,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...
1,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,[gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,NA18522,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,GM18522,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...
10,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,[gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,NA18747,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,GM18747,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...
100,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,[gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,HG01252,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,HG01252,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...
101,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,[gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,HG01252,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,HG01252,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...


## Read NTSM Output & Write To DataFrame

In [4]:
ntsm_df['ntsm_score'] = np.nan
ntsm_df['result']     = np.nan

for index, row in ntsm_df.iterrows():

        sample_ntsm_fp = row['ntsm_eval_out']
        sample_ntsm_df = pd.read_csv(sample_ntsm_fp, header=None, sep='\t')

        ntsm_df.loc[index,'ntsm_score'] = sample_ntsm_df.iloc[0][2]
        ntsm_df.loc[index,'result'] = sample_ntsm_df[3].astype('str')[0]



In [5]:
## How many rows don't match? (Should be 0)
sum(ntsm_df['result'] != 'Similar')

0

# Extract ReadStats Data

## Read in ReadStats Data Table

In [6]:
covstats_df = tp.table_to_dataframe("covstats", workspace=WORKSPACE, workspace_namespace=PROJECT)

covstats_df.head()

,ONT_seq_summ,fail_summary_stats,pass_summary_stats,sample
covstats_id,,,,
0,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,GM18522
1,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,GM18522
10,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,GM18747
100,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,HG01252
101,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,HG01252


## Read ReadStats Output & Write To DataFrame

In [7]:
summary_files = list(covstats_df['pass_summary_stats']) + list(covstats_df['fail_summary_stats'])
#summary_files=['gs://fc-72c79fce-944a-4bf1-a8d1-717ecd7a29a3/submissions/84baaeda-5840-4d9b-93da-8c6ede10596d/run_calc_ont_stats/e1842c56-7bc2-46ad-9196-62deb717a086/call-calc_ont_summary_stats/cacheCopy/glob-f0d314809f0b58a96bee1f8f36b45ca1/12_08_21_R941_HG00423_1_Guppy_6.4.6_450bps_modbases_5mc_cg_sup_prom.pass_summary_stats.txt','gs://fc-72c79fce-944a-4bf1-a8d1-717ecd7a29a3/submissions/84baaeda-5840-4d9b-93da-8c6ede10596d/run_calc_ont_stats/129377bc-f790-44b9-b6a0-ab8bbba9ec0d/call-calc_ont_summary_stats/glob-f0d314809f0b58a96bee1f8f36b45ca1/07_20_21_R941_HG02698_3_Guppy_6.5.7_450bps_modbases_5mc_cg_sup_prom.pass_summary_stats.txt','gs://fc-72c79fce-944a-4bf1-a8d1-717ecd7a29a3/submissions/84baaeda-5840-4d9b-93da-8c6ede10596d/run_calc_ont_stats/4e5a1164-d9e0-48d3-bed1-d3c25710c32b/call-calc_ont_summary_stats/attempt-2/glob-f0d314809f0b58a96bee1f8f36b45ca1/08_17_21_R941_HG02735_1_Guppy_6.5.7_450bps_modbases_5mc_cg_sup_prom.pass_summary_stats.txt', 'gs://fc-72c79fce-944a-4bf1-a8d1-717ecd7a29a3/submissions/84baaeda-5840-4d9b-93da-8c6ede10596d/run_calc_ont_stats/304ebe1c-7c06-4b5d-a9ed-c898f29fcf95/call-calc_ont_summary_stats/attempt-2/glob-f0d314809f0b58a96bee1f8f36b45ca1/08_17_21_R941_HG02735_2_Guppy_6.5.7_450bps_modbases_5mc_cg_sup_prom.pass_summary_stats.txt']
#print(summary_files)
summary_df = pd.DataFrame()

# Loop through each summary file and append its contents to the combined dataframe
for summary in summary_files:
    df = pd.read_csv(summary, sep='\t')
    #df['pass_summary_stats'] = summary
    #df['sample'] = df['File'].str.split(pat="_").str[4] 
    df.insert(1, 'sample', df['File'].str.split(pat="_").str[4])
    df.insert(2, 'flowcell', df['File'].str.split(pat="_").str[4:6].str.join('_'))
    df['File'] = df['File'].str.replace('\[\'', '').str.replace('\'\]', '').str.replace('txt','bam')
    summary_df = pd.concat([summary_df, df])

summary_df.head()

,File,sample,flowcell,read_N50,Gb,coverage,100kb+,200kb+,300kb+,400kb+,500kb+,1Mb+,whales
0,05_17_22_R941_GM18522_1_Guppy_6.5.7_450bps_mod...,GM18522,GM18522_1,90186,76.64,23.22,10.33,2.31,0.34,0.05,0.01,0.0,0
0,05_17_22_R941_GM18522_2_Guppy_6.5.7_450bps_mod...,GM18522,GM18522_2,92175,97.92,29.67,13.52,3.04,0.46,0.07,0.01,0.0,0
0,06_28_22_R941_GM18747_2_Guppy_6.5.7_450bps_mod...,GM18747,GM18747_2,83080,80.89,24.51,10.02,2.40,0.41,0.06,0.01,0.0,1
0,02_08_22_R941_HG01252_1_Guppy_6.5.7_450bps_mod...,HG01252,HG01252_1,80723,46.52,14.10,5.27,0.89,0.19,0.04,0.01,0.0,0
0,02_08_22_R941_HG01252_2_Guppy_6.5.7_450bps_mod...,HG01252,HG01252_2,80454,49.73,15.07,5.63,1.02,0.23,0.07,0.03,0.0,4


In [8]:
summary_df.shape
len(covstats_df['sample'].unique())

69

In [9]:
# sum coverage by sample
for sample in (summary_df['sample'].unique()):
    total_coverage = summary_df.loc[summary_df['sample'] == sample, 'coverage'].sum()
    if total_coverage < 60:
        print(sample, round(total_coverage,2))
# this should output nothing 

HG02015 52.93
HG02056 57.4
HG02129 55.91
HG03195 57.77
GM20805 56.25
GM20858 53.44


In [10]:
# sum coverage by sample
for sample in (summary_df['sample'].unique()):
    total_100kb_coverage = summary_df.loc[summary_df['sample'] == sample, '100kb+'].sum()
    if total_100kb_coverage < 30:
        print(sample, round(total_100kb_coverage,2))
# this should output nothing 

HG01252 22.94
HG01261 25.17
HG01530 22.73
HG01784 27.73
HG01960 19.78
HG02015 25.35
HG02056 22.51
HG02129 20.96
GM18983 29.98
HG02391 23.87
HG02841 27.41
GM19043 24.57
HG03130 25.44
HG03139 29.41
HG03195 21.53
HG03742 24.75
HG03792 20.94
GM19087 25.93
GM19120 28.32
GM19159 27.43
GM19185 23.19
GM20799 20.63
GM20805 21.13
GM20858 20.09
GM20905 24.38
HG00099 23.2
HG00280 22.72
HG00323 25.71


# Check tables, then export to tsv

In [12]:
summary_df.head()

,File,sample,flowcell,read_N50,Gb,coverage,100kb+,200kb+,300kb+,400kb+,500kb+,1Mb+,whales
0,05_17_22_R941_GM18522_1_Guppy_6.5.7_450bps_mod...,GM18522,GM18522_1,90186,76.64,23.22,10.33,2.31,0.34,0.05,0.01,0.0,0
0,05_17_22_R941_GM18522_2_Guppy_6.5.7_450bps_mod...,GM18522,GM18522_2,92175,97.92,29.67,13.52,3.04,0.46,0.07,0.01,0.0,0
0,06_28_22_R941_GM18747_2_Guppy_6.5.7_450bps_mod...,GM18747,GM18747_2,83080,80.89,24.51,10.02,2.40,0.41,0.06,0.01,0.0,1
0,02_08_22_R941_HG01252_1_Guppy_6.5.7_450bps_mod...,HG01252,HG01252_1,80723,46.52,14.10,5.27,0.89,0.19,0.04,0.01,0.0,0
0,02_08_22_R941_HG01252_2_Guppy_6.5.7_450bps_mod...,HG01252,HG01252_2,80454,49.73,15.07,5.63,1.02,0.23,0.07,0.03,0.0,4


In [13]:
ntsm_df.head()

,ntsv_count_2,fastq_list,ntsm_eval_out,read_2_fastq,1000G,read_1_fastq,sample,ONT_pass_bam,ntsv_count_1,ntsm_score,result
ntsm_id,,,,,,,,,,,
0,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,[gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,NA18522,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,GM18522,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,0.336724,Similar
1,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,[gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,NA18522,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,GM18522,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,0.333979,Similar
10,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,[gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,NA18747,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,GM18747,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,0.340225,Similar
100,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,[gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,HG01252,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,HG01252,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,0.331159,Similar
101,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,[gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,HG01252,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,HG01252,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,gs://fc-93c439d3-8a8a-4245-bc5c-f23140dcf2cf/s...,0.339225,Similar


In [14]:
# Create table
outSumm = os.path.join(bucket, WORKSPACE + '_summary.tsv')
summary_df.to_csv(outSumm, sep="\t", index=False)

In [15]:
# Create table
outNTSM = os.path.join(bucket, WORKSPACE + '_NTSM.tsv')
ntsm_df.to_csv(outNTSM, sep="\t", index=False)